참고 링크 : https://github.com/puzzlecollector/DACON-fake-news-identification/blob/main/lightgbm%20more%20features.ipynb
- Dacon 가짜뉴스경진대회에 참여하였는데, 모델링에 대해 많은 고민을 했음에도 불구하고 점수가 잘 나오지 않아 상위권 및 수상자분들의 코드가 궁금해서 github를 통해 찾아보게 되어 발견한 링크의 __Code Review__

In [1]:
import numpy as np 
import pandas as pd 
import tqdm
import string

train = pd.read_csv('news_train.csv')
test = pd.read_csv('news_test.csv') 
submission = pd.read_csv('sample_submission.csv')

In [2]:
import seaborn as sns
#import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes

In [4]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

import nltk # for stopwords 
from nltk.corpus import stopwords
import gensim # for Word2Vec embeddings 
from sklearn.feature_extraction.text import CountVectorizer

from eunjeon import Mecab

##### <center>Extract Meta Features</center>

In [5]:
mecab = Mecab()

In [53]:
# Mecab 라이브러리를 활용한 형태소 분석
train['title_tokenized'] = train['title'].apply(lambda x: mecab.morphs(x))
train['content_tokenized'] = train['content'].apply(lambda x: mecab.morphs(x))

test['title_tokenized'] = test['title'].apply(lambda x: mecab.morphs(x))
test['content_tokenized'] = test['content'].apply(lambda x: mecab.morphs(x))

##### number of words in title (2차원)

In [7]:
# number of words in title
train['num_words_title'] = train['title_tokenized'].apply(lambda x: len(x)) 
test['num_words_title'] = test['title_tokenized'].apply(lambda x: len(x))   

# number of words in content 
train['num_words_content'] = train['content_tokenized'].apply(lambda x: len(x)) 
test['num_words_content'] = test['content_tokenized'].apply(lambda x: len(x))

train.tail(2)

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11


##### number of unique words in title (2차원)

In [8]:
# number of unique words in title 
train['title_num_unique'] = train['title_tokenized'].apply(lambda x: len(set(x))) 
test['title_num_unique'] = test['title_tokenized'].apply(lambda x: len(set(x))) 

# number of unique words in content 
train['content_num_unique'] = train['content_tokenized'].apply(lambda x: len(set(x))) 
test['content_num_unique'] = test['content_tokenized'].apply(lambda x: len(set(x)))

train.tail(2)

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,title_num_unique,content_num_unique
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,10,11
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,10,11


##### number of punctuations in text  (2차원)

In [9]:
# number of punctuations in text 
train['num_punctuations_title'] = train['title_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))
test['num_punctuations_title'] = test['title_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))

train['num_punctuations_content'] = train['content_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))
test['num_punctuations_content'] = test['content_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))

train.tail(2)

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,title_num_unique,content_num_unique,num_punctuations_title,num_punctuations_content
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,10,11,3,0
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,10,11,3,0


##### 특수문자로 시작하는지에 대한 여부 (6차원)
1. [ 또는 (
2. ] 또는 )
3. ' 로 시작
4. ' 로 끝남
5. 숫자로 시작
6. 숫자로 끝남

In [10]:
# [ 또는 (로 시작하는지에 대한 여부 
train["title_startswith_["]=train.title.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(")) 
train["content_startswith_["]=train.content.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(") ) 
test["title_startswith_["]=test.title.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(")) 
test["content_startswith_["]=test.content.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(") ) 


# ] 또는 )로 시작하는지에 대한 여부 
train["title_endswith_]"]=train.title.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")"))
train["content_endswith_]"]=train.content.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")") )
test["title_endswith_]"]=test.title.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")"))
test["content_endswith_]"]=test.content.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")") )


# ' 로 시작하는지에 대한 여부 
train["title_startswith_quote"]=train.title.apply(lambda x : str(x).startswith('"'))
train["content_startswith_quote"]=train.content.apply(lambda x : str(x).startswith('"'))
test["title_startswith_quote"]=test.title.apply(lambda x : str(x).startswith('"'))
test["content_startswith_quote"]=test.content.apply(lambda x : str(x).startswith('"'))


# '로 끝나는지에 대한 여부
train["title_endswith_quote"]=train.title.apply(lambda x : str(x).endswith('"'))
train["content_endswith_quote"]=train.content.apply(lambda x : str(x).endswith('"'))
test["title_endswith_quote"]=test.title.apply(lambda x : str(x).endswith('"'))
test["content_endswith_quote"]=test.content.apply(lambda x : str(x).endswith('"'))


# 숫자로 시작하는지에 대한 여부 
train["title_startswith_number"]=train.title.apply(lambda x : str(x)[0].isdigit())
train["content_startswith_number"]=train.content.apply(lambda x : str(x)[0].isdigit())
test["title_startswith_number"]=test.title.apply(lambda x : str(x)[0].isdigit())
test["content_startswith_number"]=test.content.apply(lambda x : str(x)[0].isdigit())


# 숫자로 끝나는지에 대한 여부 
train["title_endswith_number"]=train.title.apply(lambda x : str(x)[-1].isdigit())
train["content_endswith_number"]=train.content.apply(lambda x : str(x)[-1].isdigit())
test["title_endswith_number"]=test.title.apply(lambda x : str(x)[-1].isdigit())
test["content_endswith_number"]=test.content.apply(lambda x : str(x)[-1].isdigit())

train.tail(2)

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,...,title_endswith_],content_endswith_],title_startswith_quote,content_startswith_quote,title_endswith_quote,content_endswith_quote,title_startswith_number,content_startswith_number,title_endswith_number,content_endswith_number
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,False,False,False,False,False,False,False,True,False,False
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,False,False,False,False,False,False,False,True,False,False


##### title, content length (2 차원)

In [11]:
# title length 
train["title_length"] = train['title'].apply(lambda x : len(x))
test["title_length"] = test['title'].apply(lambda x : len(x))

# content length 
train['content_length'] = train['content'].apply(lambda x: len(x))
test['content_length'] = test['content'].apply(lambda x: len(x))

##### 제목에 등장하는 단어들의 평균 길이 ( 1 차원)

In [13]:
# average length of word in title 
train['title_mean_length'] = train['title_tokenized'].apply(lambda x: np.mean([len(w) for w in x])) 
test['title_mean_length'] = test['title_tokenized'].apply(lambda x: np.mean([len(w) for w in x]))

display(train.tail(2))
print(train.shape, test.shape)

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,...,content_startswith_quote,title_endswith_quote,content_endswith_quote,title_startswith_number,content_startswith_number,title_endswith_number,content_endswith_number,title_length,content_length,title_mean_length
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,False,False,False,False,True,False,False,24,27,1.9
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,False,False,False,False,True,False,False,24,27,1.9


(118745, 29) (142565, 29)


##### Extract Text Based Features
- Extract TF-IDF 1-3 ngram features, then apply truncated SVD to incorporate into our dataframe

In [14]:
def mecab_tokenizer(text):  
    tokens_mecab = mecab.morphs(text) 
    return tokens_mecab

In [15]:
# used for singular value decomposition  
tfidf_vec = TfidfVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
full_tfidf_title = tfidf_vec.fit_transform(train['title'].values.tolist() + test['title'].values.tolist())
train_tfidf_title = tfidf_vec.transform(train['title'].values.tolist()) 
test_tfidf_title = tfidf_vec.transform(test['title'].values.tolist())

In [16]:
full_tfidf_title.shape, train_tfidf_title.shape, test_tfidf_title.shape

((261310, 170510), (118745, 170510), (142565, 170510))

In [17]:
# used for singular value decomposition  
tfidf_vec = TfidfVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
full_tfidf_content = tfidf_vec.fit_transform(train['content'].values.tolist() + test['content'].values.tolist())
train_tfidf_content = tfidf_vec.transform(train['content'].values.tolist()) 
test_tfidf_content = tfidf_vec.transform(test['content'].values.tolist())

In [18]:
full_tfidf_content.shape, train_tfidf_content.shape, test_tfidf_content.shape

((261310, 2461160), (118745, 2461160), (142565, 2461160))

##### 제목과 내용을 통해 TF-IDF 학습
- __제목 차원 TruncatedSVD 를 통한 20차원으로 축소__

In [19]:
#title
svd_obj = TruncatedSVD(n_components = 20, algorithm = 'arpack') 
svd_obj.fit(full_tfidf_title)
train_svd_title = pd.DataFrame(svd_obj.transform(train_tfidf_title)) 
test_svd_title = pd.DataFrame(svd_obj.transform(test_tfidf_title))

In [20]:
train_svd_title.columns = ['svd_word_title_' + str(i) for i in range(20)] 
test_svd_title.columns = ['svd_word_title_' + str(i) for i in range(20)] 

train = pd.concat([train, train_svd_title], axis = 1) 
test = pd.concat([test, test_svd_title], axis = 1)  

display(train_svd_title.tail(3))
display(test_svd_title.tail(3))

,svd_word_title_0,svd_word_title_1,svd_word_title_2,svd_word_title_3,svd_word_title_4,svd_word_title_5,svd_word_title_6,svd_word_title_7,svd_word_title_8,svd_word_title_9,svd_word_title_10,svd_word_title_11,svd_word_title_12,svd_word_title_13,svd_word_title_14,svd_word_title_15,svd_word_title_16,svd_word_title_17,svd_word_title_18,svd_word_title_19
118742,0.015566,0.034635,0.00185,0.051394,0.000008,-0.002585,0.005632,-0.009827,0.028778,0.016189,-0.014199,0.001359,0.010644,-0.001327,0.009217,0.004156,0.00856,-0.021201,0.000703,0.017163
118743,0.015566,0.034635,0.00185,0.051394,0.000008,-0.002585,0.005632,-0.009827,0.028778,0.016189,-0.014199,0.001359,0.010644,-0.001327,0.009217,0.004156,0.00856,-0.021201,0.000703,0.017163
118744,0.015566,0.034635,0.00185,0.051394,0.000008,-0.002585,0.005632,-0.009827,0.028778,0.016189,-0.014199,0.001359,0.010644,-0.001327,0.009217,0.004156,0.00856,-0.021201,0.000703,0.017163


,svd_word_title_0,svd_word_title_1,svd_word_title_2,svd_word_title_3,svd_word_title_4,svd_word_title_5,svd_word_title_6,svd_word_title_7,svd_word_title_8,svd_word_title_9,svd_word_title_10,svd_word_title_11,svd_word_title_12,svd_word_title_13,svd_word_title_14,svd_word_title_15,svd_word_title_16,svd_word_title_17,svd_word_title_18,svd_word_title_19
142562,0.539313,-0.156937,-0.002495,-0.071633,-0.000819,0.006111,0.00054,-0.008687,0.033944,0.00623,-0.03586,0.013994,-0.00279,0.005082,-0.000707,-0.001095,-0.001463,0.009377,-0.004433,-0.019869
142563,0.539313,-0.156937,-0.002495,-0.071633,-0.000819,0.006111,0.00054,-0.008687,0.033944,0.00623,-0.03586,0.013994,-0.00279,0.005082,-0.000707,-0.001095,-0.001463,0.009377,-0.004433,-0.019869
142564,0.539313,-0.156937,-0.002495,-0.071633,-0.000819,0.006111,0.00054,-0.008687,0.033944,0.00623,-0.03586,0.013994,-0.00279,0.005082,-0.000707,-0.001095,-0.001463,0.009377,-0.004433,-0.019869


- __내용 차원 TruncatedSVD 를 통한 20차원으로 축소__

In [21]:
#content
svd_obj = TruncatedSVD(n_components = 20, algorithm = 'arpack') 
svd_obj.fit(full_tfidf_content) 
train_svd_content = pd.DataFrame(svd_obj.transform(train_tfidf_content)) 
test_svd_content = pd.DataFrame(svd_obj.transform(test_tfidf_content))

In [22]:
train_svd_content.columns = ['svd_word_content_' + str(i) for i in range(20)] 
test_svd_content.columns = ['svd_word_content_' + str(i) for i in range(20)] 

train = pd.concat([train, train_svd_content], axis = 1) 
test = pd.concat([test, test_svd_content], axis = 1)  

display(train_svd_content.tail(3))
display(test_svd_content.tail(3))
# del full_tfidf_content, train_tfidf_content, test_tfidf_content, train_svd_content, test_svd_content

,svd_word_content_0,svd_word_content_1,svd_word_content_2,svd_word_content_3,svd_word_content_4,svd_word_content_5,svd_word_content_6,svd_word_content_7,svd_word_content_8,svd_word_content_9,svd_word_content_10,svd_word_content_11,svd_word_content_12,svd_word_content_13,svd_word_content_14,svd_word_content_15,svd_word_content_16,svd_word_content_17,svd_word_content_18,svd_word_content_19
118742,0.426599,0.017700,-0.352812,0.518868,-0.205959,-0.158608,0.531105,-0.242649,-0.069693,-0.036576,-0.075056,-0.050952,-0.010171,-0.007523,-0.019699,0.004028,0.011081,-0.001630,-0.002340,0.006837
118743,0.218511,-0.122412,0.562394,0.638329,0.256485,0.077538,-0.284790,-0.096131,-0.041001,0.205266,0.076371,-0.036103,-0.006039,-0.003042,-0.006499,0.001195,-0.012723,-0.000328,0.001796,-0.001010
118744,0.218511,-0.122412,0.562394,0.638329,0.256485,0.077538,-0.284790,-0.096131,-0.041001,0.205266,0.076371,-0.036103,-0.006039,-0.003042,-0.006499,0.001195,-0.012723,-0.000328,0.001796,-0.001010


,svd_word_content_0,svd_word_content_1,svd_word_content_2,svd_word_content_3,svd_word_content_4,svd_word_content_5,svd_word_content_6,svd_word_content_7,svd_word_content_8,svd_word_content_9,svd_word_content_10,svd_word_content_11,svd_word_content_12,svd_word_content_13,svd_word_content_14,svd_word_content_15,svd_word_content_16,svd_word_content_17,svd_word_content_18,svd_word_content_19
142562,0.426599,0.017700,-0.352812,0.518868,-0.205959,-0.158608,0.531105,-0.242649,-0.069693,-0.036576,-0.075056,-0.050952,-0.010171,-0.007523,-0.019699,0.004028,0.011081,-0.001630,-0.002340,0.006837
142563,0.218511,-0.122412,0.562394,0.638329,0.256485,0.077538,-0.284790,-0.096131,-0.041001,0.205266,0.076371,-0.036103,-0.006039,-0.003042,-0.006499,0.001195,-0.012723,-0.000328,0.001796,-0.001010
142564,0.218511,-0.122412,0.562394,0.638329,0.256485,0.077538,-0.284790,-0.096131,-0.041001,0.205266,0.076371,-0.036103,-0.006039,-0.003042,-0.006499,0.001195,-0.012723,-0.000328,0.001796,-0.001010


In [23]:
train.shape, test.shape

((118745, 69), (142565, 69))

In [24]:
# delete unnecessary dataframes 
del train_svd_title, test_svd_title, train_svd_content, test_svd_content, full_tfidf_title, full_tfidf_content

##### Extract Count vectorizer 1-3 ngram, then fit MNB  to add to our dataframe ( 8 차원)
- predict_proba (True or False)
1. MNB for tf-idf title matrix (2차원) 
2. MNB for tfidf content matrix (2차원)
3. MNB for tf-idf title matrix (2차원)
4. MNB for tf-idf content matrix (2차원)

In [25]:
#title
cvec_full = CountVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
cvec_full.fit(train['title'].values.tolist() + test['title'].values.tolist()) 
train_cvec_title = cvec_full.transform(train['title'].values.tolist()) 
test_cvec_title = cvec_full.transform(test['title'].values.tolist())

C:\Users\user654\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [26]:
train_cvec_title.shape, test_cvec_title.shape

((118745, 170510), (142565, 170510))

In [27]:
#content
cvec_full_content = CountVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
cvec_full_content.fit(train['content'].values.tolist() + test['content'].values.tolist()) 
train_cvec_content = cvec_full_content.transform(train['content'].values.tolist()) 
test_cvec_content = cvec_full_content.transform(test['content'].values.tolist())

In [28]:
train_cvec_content.shape, test_cvec_content.shape

((118745, 2461160), (142565, 2461160))

In [29]:
def runMNB(train_x, train_y, test_x, test_y, test_x2):  
    model = MultinomialNB() #모델 생성
    model.fit(train_x, train_y) #모델 학습
    pred_test_y = model.predict_proba(test_x) # Y에 대한 예측 확률 output 
    pred_test_y2 = model.predict_proba(test_x2)  # 1 - pred_test_y (binary classifcation)
    return pred_test_y, pred_test_y2, model

In [30]:
y_train = train['info'] 
y_train = np.asarray(y_train)
y_train.shape

(118745,)

##### 1.  MNB : Using tf-idf title matrix


In [40]:
cv_scores = [] 
pred_full_test = 0 
pred_train = np.zeros([train.shape[0],2]) #학습데이터 (2 X nrow) Matrix
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 990101)  #5- Fold 무작위 index 추출
 
for train_idx, val_idx in kf.split(train, y_train): #kfold 를 위한 , for loop
    train_x, val_x = train_cvec_title[train_idx], train_cvec_title[val_idx] 
    train_y, val_y = y_train[train_idx], y_train[val_idx] 
    pred_val_y, pred_test_y, model = runMNB(train_x, train_y, val_x, val_y, test_cvec_title) 
    pred_full_test = pred_full_test + pred_test_y 
    pred_train[val_idx,:] = pred_val_y   
    
    
pred_full_test /= 5.0  #교차검증에 대한 평균값

In [41]:
train['nb_tfidf_real_title'] = pred_train[:,0] 
train['nb_tfidf_fake_title'] = pred_train[:,1] 
test['nb_tfidf_real_title'] = pred_full_test[:,0] 
test['nb_tfidf_fake_title'] = pred_full_test[:,1]

##### 2.  MNB : Using tf-idf content matrix

In [42]:
cv_scores = [] 
pred_full_test = 0 
pred_train = np.zeros([train.shape[0],2]) 
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 990101) 

for train_idx, val_idx in kf.split(train, y_train):
    train_x, val_x = train_tfidf_content[train_idx], train_tfidf_content[val_idx] 
    train_y, val_y = y_train[train_idx], y_train[val_idx] 
    pred_val_y, pred_test_y, model = runMNB(train_x, train_y, val_x, val_y, test_tfidf_content) 
    pred_full_test = pred_full_test + pred_test_y 
    pred_train[val_idx,:] = pred_val_y   
    
pred_full_test /= 5.0

In [43]:
train['nb_tfidf_real_content'] = pred_train[:,0] 
train['nb_tfidf_fake_content'] = pred_train[:,1] 
test['nb_tfidf_real_content'] = pred_full_test[:,0]  
test['nb_tfidf_fake_content'] = pred_full_test[:,1]

display(train.tail(3))
print(train.shape, test.shape)

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,...,svd_word_content_18,svd_word_content_19,nb_cvec_real_title,nb_cvec_fake_title,nb_tfidf_real_content,nb_tfidf_fake_content,nb_cvec_real_content,nb_cvec_fake_content,nb_tfidf_real_title,nb_tfidf_fake_title
118742,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,50,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[똑똑, 해진, 소비자, ., ., 한국, 도, 이젠, 소형차, 시대]",10,10,...,-0.002340,0.006837,2.874338e-71,1.0,3.236437e-12,1.0,2.874338e-71,1.0,9.606627e-17,1.0
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,0.001796,-0.001010,3.408893e-69,1.0,1.243329e-11,1.0,3.408893e-69,1.0,1.718020e-20,1.0
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,0.001796,-0.001010,2.732797e-69,1.0,1.227173e-11,1.0,2.732797e-69,1.0,3.754990e-16,1.0


(118745, 77) (142565, 77)


##### 3.  MNB : Using tf-idf title matrix

In [44]:
cv_scores = [] 
pred_full_test = 0 
pred_train = np.zeros([train.shape[0],2]) 
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 990101)

for train_idx, val_idx in kf.split(train, y_train):
    train_x, val_x = train_cvec_title[train_idx], train_cvec_title[val_idx] 
    train_y, val_y = y_train[train_idx], y_train[val_idx] 
    pred_val_y, pred_test_y, model = runMNB(train_x, train_y, val_x, val_y, test_cvec_title) 
    pred_full_test = pred_full_test + pred_test_y 
    pred_train[val_idx,:] = pred_val_y   
    
pred_full_test /= 5.0

In [45]:
train['nb_cvec_real_title'] = pred_train[:,0] 
train['nb_cvec_fake_title'] = pred_train[:,1] 
test['nb_cvec_real_title'] = pred_full_test[:,0]  
test['nb_cvec_fake_title'] = pred_full_test[:,1]

display(train.tail(3))

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,...,svd_word_content_18,svd_word_content_19,nb_cvec_real_title,nb_cvec_fake_title,nb_tfidf_real_content,nb_tfidf_fake_content,nb_cvec_real_content,nb_cvec_fake_content,nb_tfidf_real_title,nb_tfidf_fake_title
118742,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,50,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[똑똑, 해진, 소비자, ., ., 한국, 도, 이젠, 소형차, 시대]",10,10,...,-0.002340,0.006837,9.606627e-17,1.0,3.236437e-12,1.0,2.874338e-71,1.0,9.606627e-17,1.0
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,0.001796,-0.001010,1.718020e-20,1.0,1.243329e-11,1.0,3.408893e-69,1.0,1.718020e-20,1.0
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,0.001796,-0.001010,3.754990e-16,1.0,1.227173e-11,1.0,2.732797e-69,1.0,3.754990e-16,1.0


##### 4. MNB : Using tf-idf content matrix

In [46]:
cv_scores = [] 
pred_full_test = 0 
pred_train = np.zeros([train.shape[0],2]) 
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 990101)

for train_idx, val_idx in kf.split(train, y_train):
    train_x, val_x = train_cvec_content[train_idx], train_cvec_content[val_idx]
    train_y, val_y = y_train[train_idx], y_train[val_idx]
    pred_val_y, pred_test_y, model = runMNB(train_x, train_y, val_x, val_y, test_cvec_content)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_idx,:] = pred_val_y
    
pred_full_test /= 5.0

In [47]:
train['nb_cvec_real_content'] = pred_train[:,0] 
train['nb_cvec_fake_content'] = pred_train[:,1] 
test['nb_cvec_real_content'] = pred_full_test[:,0]  
test['nb_cvec_fake_content'] = pred_full_test[:,1]

display(train.tail(3))
print("Final shape of the dataframe") 
train.shape, test.shape

,n_id,date,title,content,ord,info,title_tokenized,content_tokenized,num_words_title,num_words_content,...,svd_word_content_18,svd_word_content_19,nb_cvec_real_title,nb_cvec_fake_title,nb_tfidf_real_content,nb_tfidf_fake_content,nb_cvec_real_content,nb_cvec_fake_content,nb_tfidf_real_title,nb_tfidf_fake_title
118742,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,50,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[똑똑, 해진, 소비자, ., ., 한국, 도, 이젠, 소형차, 시대]",10,10,...,-0.002340,0.006837,9.606627e-17,1.0,3.236437e-12,1.0,2.874338e-71,1.0,9.606627e-17,1.0
118743,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,51,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,0.001796,-0.001010,1.718020e-20,1.0,1.243329e-11,1.0,3.408893e-69,1.0,1.718020e-20,1.0
118744,NEWS01333,20200117,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,52,1,"[모나리자, ,, 중원, 지분, 16, ., 88, %, 양수, 결정]","[2020, 년, 한국, TV, 2, 대중, 1, 대, 인터넷, 연결, 된다]",10,11,...,0.001796,-0.001010,3.754990e-16,1.0,1.227173e-11,1.0,2.732797e-69,1.0,3.754990e-16,1.0


Final shape of the dataframe


((118745, 77), (142565, 77))

## <center>Train LightGBM Model</center>


In [58]:
# 불필요한 Column 제거
x_train = train.drop(['n_id','info','title','content','title_tokenized','content_tokenized'], axis = 1) 
y_train = train['info'] 
x_test = test.drop(['n_id','id','title','content','title_tokenized','content_tokenized'], axis = 1)


#최종 학습에 사용될 데이터셋 (nrows X 71) Matrix
display(x_train.tail(3))
print(x_train.shape, y_train.shape , x_test.shape)

,date,ord,num_words_title,num_words_content,title_num_unique,content_num_unique,num_punctuations_title,num_punctuations_content,title_startswith_[,content_startswith_[,...,svd_word_content_18,svd_word_content_19,nb_cvec_real_title,nb_cvec_fake_title,nb_tfidf_real_content,nb_tfidf_fake_content,nb_cvec_real_content,nb_cvec_fake_content,nb_tfidf_real_title,nb_tfidf_fake_title
118742,20200117,50,10,10,10,9,3,2,False,False,...,-0.002340,0.006837,9.606627e-17,1.0,3.236437e-12,1.0,2.874338e-71,1.0,9.606627e-17,1.0
118743,20200117,51,10,11,10,11,3,0,False,False,...,0.001796,-0.001010,1.718020e-20,1.0,1.243329e-11,1.0,3.408893e-69,1.0,1.718020e-20,1.0
118744,20200117,52,10,11,10,11,3,0,False,False,...,0.001796,-0.001010,3.754990e-16,1.0,1.227173e-11,1.0,2.732797e-69,1.0,3.754990e-16,1.0


(118745, 71) (118745,) (142565, 71)


In [63]:
x_train = np.asarray(x_train) 
y_train = np.asarray(y_train) 
x_test = np.asarray(x_test) 

x_train.shape, y_train.shape, x_test.shape

((118745, 71), (118745,), (142565, 71))

In [ ]:
# # save dataframe 
# np.save('x_train_latest.npy',x_train) 
# np.save('y_train_latest.npy',y_train) 
# np.save('x_test_latest.npy',x_test)

In [66]:
k = 5 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 990101) 
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)): 
    train_x, val_x = x_train[train_idx], x_train[val_idx]
    train_y, val_y = y_train[train_idx], y_train[val_idx] 
    
    params = {'learning_rate': 0.01,
              'max_depth': 16, 
              'objective': 'binary',
              'metric': 'binary_logloss',
              'is_training_metric': True,
              'num_leaves': 128,
              'feature_fraction': 0.9,
              'bagging_fraction': 0.75, 
              'bagging_freq': 5,
              'seed': 960418} 
    
    train_ds = lgbm.Dataset(train_x, label = train_y) 
    val_ds = lgbm.Dataset(val_x, label = val_y) 
    model = lgbm.train(params, train_ds, 1000, val_ds, verbose_eval = 200, early_stopping_rounds = 100) 
    models.append(model)

[LightGBM] [Info] Number of positive: 37546, number of negative: 57450
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011735 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13498
[LightGBM] [Info] Number of data points in the train set: 94996, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.395238 -> initscore=-0.425348
[LightGBM] [Info] Start training from score -0.425348
Training until validation scores don't improve for 100 rounds
[200]	valid_0's binary_logloss: 0.0825168
[400]	valid_0's binary_logloss: 0.0226684
[600]	valid_0's binary_logloss: 0.0132334
[800]	valid_0's binary_logloss: 0.0113586
[1000]	valid_0's binary_logloss: 0.0110289
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.0110289
[LightGBM] [Info] Number of positive: 37546, number of negative: 57450
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the ov

In [67]:
pred1 = models[0].predict(x_test) 
pred2 = models[1].predict(x_test) 
pred3 = models[2].predict(x_test) 
pred4 = models[3].predict(x_test) 
pred5 = models[4].predict(x_test)

pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5)/5.0

class_pred = np.where(pred_avg > 0.5, 1, 0).reshape(-1)

submission['info'] = class_pred

In [68]:
# 최종 제출용 양식
submission

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
